this using https://github.com/NVlabs/stylegan2-ada-pytorch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

%cd stylegan2-ada-pytorch/

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 1.12 MiB | 33.80 MiB/s, done.
Resolving deltas: 100% (55/55), done.
     |████████████████████████████████| 112kB 29.7MB/s 
/content/stylegan2-ada-pytorch


In [ ]:
%cp ../drive/MyDrive/network-snapshot-000200.pkl ../

In [ ]:
!mkdir "../dataset"
!mkdir "../dataset/resized_images"

In [ ]:
!python train.py --help

In [ ]:
%rm -rf ../drive/MyDrive/dataset/

In [3]:
%cp -r ../drive/MyDrive/dataset ../

In [ ]:
import glob
from PIL import Image, ImageChops

count = 0

for path in glob.glob("../drive/MyDrive/nikos/*.*"):

  count += 1

  size = (128,128)
  image = Image.open(path)
  image.thumbnail(size, Image.ANTIALIAS)
  image_size = image.size

  thumb = image.crop( (0, 0, size[0], size[1]) )

  offset_x = max( (size[0] - image_size[0]) / 2, 0 )
  offset_y = max( (size[1] - image_size[1]) / 2, 0 )

  thumb = ImageChops.offset(thumb, int(offset_x), int(offset_y)).convert("RGB")

  thumb.save(f"../dataset/resized_images/img{count}.png")

  if count % 100 == 0:
    print(count)

print(count)

In [5]:
!python dataset_tool.py --source ../dataset/resized_images --dest ../dataset/train_images

100% 1195/1195 [00:02<00:00, 580.11it/s]


In [ ]:
!python train.py --outdir ../result --data ../dataset/train_images/ --kimg 600


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 50,
  "network_snapshot_ticks": 50,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "../dataset/train_images/",
    "use_labels": false,
    "max_size": 1195,
    "xflip": false,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 51

In [ ]:
%cp ../result/00001--auto1-kimg1000-resumecustom/*.pkl ../drive/MyDrive

In [ ]:
!python generate.py --outdir=../out --seeds=200-505 --network=../network-snapshot-000400.pkl

import glob
from IPython.display import Image
from PIL import Image
import matplotlib.pyplot as plt



for path in glob.glob("../out/*.*"):
    plt.figure()
    plt.imshow(Image.open(path))
